# Using Parameter Estimation with Modular Property Packages

## 1. Introduction

This Jupyter Notebook estimates binary interaction parameters for a CO$_2$-Ionic liquid property package. A property package has been created for CO$_2$-[bmim][PF6]. We will utilize Pyomo's `parmest` tool in conjuction with IDAES models for parameter estimation. We demonstrate these tools by estimating the parameters associated with the Peng-Robinson property model for a benzene-toluene mixture. The Peng-Robinson EOS the binary interaction parameter (kappa_ij). When estimating parameters associated with the property package, IDAES provides the flexibility of doing the parameter estimation by just using the state block or by using a unit model with a specified property package. This module will demonstrate parameter estimation by using the flash unit model with a Modular Property Package.

### 1.1 Tutorial objectives

* Utilize the Modular Property Package framework, which provides a flexible platform for users to build property packages by calling upon libraries of modular sub-models to build up complex property calculations with the least effort possible.
* Set up a method to return an initialized model
* Set up the parameter estimation problem using `parmest`

## 2. Problem Statement

### 2.1 Importing Pyomo and IDAES model and flowsheet components.

In the next cell, we will be importing the necessary components from Pyomo and IDAES.

In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import Flash unit model from idaes.models.unit_models
from idaes.models.unit_models import Flash

### 2.2 Import parmest 

In [2]:
import pyomo.contrib.parmest.parmest as parmest

### 2.3 Import the Modular Property framework

In [3]:
from idaes.models.properties.modular_properties.examples.CO2_bmimPF6_PR import configuration

from idaes.models.properties.modular_properties import GenericParameterBlock

### 2.4 Import data

In the next cell, we will be importing `pandas` and the `.csv` file with preassure and composition data. For this example, we load data from the csv file CO2_IL_298.csv. The dataset consists of ninteen data points which provide the mole fraction of [bmim][PF6] and carbon dioxide and the pressure at three different temperatures.

In [4]:
import pandas as pd

# Load data from csv
data = pd.read_csv('CO2_IL_298.csv')

## 3.0 Setting up an initialized model

We need to provide a method that returns an initialized model to the `parmest` tool in Pyomo.

How we build the model will depend on the data we provided in the data dataframe from pir .csv file.

In this case we have data on the liquid mixture, the temperature and the pressure. We will fix the temperature, mole franction in the liquid phase, and the mole fraction of the inlet. 

In [5]:
def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(dynamic=False)

    m.fs.properties = GenericParameterBlock(**configuration)

    m.fs.state_block = m.fs.properties.build_state_block([1], defined_state=True)

    m.fs.state_block[1].flow_mol.fix(1)
    x = float(data["x_carbon_dioxide"])+0.5
    m.fs.state_block[1].temperature.fix(float(data["temperature"]))
    m.fs.state_block[1].pressure.fix(float(data["pressure"]))
    m.fs.state_block[1].mole_frac_comp["bmimPF6"].fix(1-x)
    m.fs.state_block[1].mole_frac_comp["carbon_dioxide"].fix(x)
    
    # parameter - kappa_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa['bmimPF6', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa['bmimPF6', 'carbon_dioxide'].fix(-0.047)
    m.fs.properties.PR_kappa['carbon_dioxide', 'carbon_dioxide'].fix(0)
    m.fs.properties.PR_kappa['carbon_dioxide', 'bmimPF6'].fix(0.002)

    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.INFO)

    # Fix the state variables on the state block
    m.fs.state_block[1].pressure.unfix()
    m.fs.state_block[1].temperature.fix(float(data["temperature"]))
    m.fs.state_block[1].mole_frac_phase_comp['Liq', 'bmimPF6'].fix(float(data["x_bmimPF6"]))
    m.fs.state_block[1].mole_frac_phase_comp['Liq', 'carbon_dioxide'].fix(float(data["x_carbon_dioxide"]))
    m.fs.state_block[1].mole_frac_comp['bmimPF6'].fix(float(data["x_bmimPF6"]))
    m.fs.state_block[1].mole_frac_comp['carbon_dioxide'].unfix()
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa['bmimPF6', 'carbon_dioxide'].setlb(-5)
    m.fs.properties.PR_kappa['bmimPF6', 'carbon_dioxide'].setub(5)

    m.fs.properties.PR_kappa['carbon_dioxide', 'bmimPF6'].setlb(-5)
    m.fs.properties.PR_kappa['carbon_dioxide', 'bmimPF6'].setub(5)

    # Return initialized flash model
    return m

### 3.1 Solving square problem

In [6]:
from idaes.core.util.model_statistics import degrees_of_freedom
import pytest

test_data = {"temperature": 298, "pressure": 812323, "x_bmimPF6":0.86, "x_carbon_dioxide":0.14}

m = PR_model(test_data)

# Check that degrees of freedom is 0
assert degrees_of_freedom(m) == 0

2022-08-25 12:01:00 [INFO] idaes.init.fs.state_block: Starting initialization
WARNING (W1002): Setting Var
    'fs.state_block[1].log_mole_frac_tbub[Vap,Liq,carbon_dioxide]' to a
    numeric value `1.3789905650578088e-06` outside the bounds (None, 0).
    See also https://pyomo.readthedocs.io/en/latest/errors.html#w1002
2022-08-25 12:01:01 [INFO] idaes.init.fs.state_block: Dew and bubble point initialization: optimal - Optimal Solution Found.
2022-08-25 12:01:01 [INFO] idaes.init.fs.state_block: Equilibrium temperature initialization completed.
2022-08-25 12:01:01 [INFO] idaes.init.fs.state_block: State variable initialization completed.
2022-08-25 12:01:01 [INFO] idaes.init.fs.state_block: Phase equilibrium initialization: optimal - Optimal Solution Found.
2022-08-25 12:01:01 [INFO] idaes.init.fs.state_block: Property initialization: optimal - Optimal Solution Found.
2022-08-25 12:01:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found

## 4.0 Parameter estimation using parmest 

### 4.1 List of variable names to be estimated

Create a list of vars to estimate

In [7]:
variable_name = ["fs.properties.PR_kappa['bmimPF6', 'carbon_dioxide']",
                 "fs.properties.PR_kappa['carbon_dioxide', 'bmimPF6']"]

### 4.2 Create method to return an expression that computes the sum of squared error

We need to provide a method to return an expression to compute the sum of squared errors that will be used as the objective in solving the parameter estimation problem. For this problem, the error will be computed for the pressure.

In [8]:
def SSE(m, data):   
    expr = ((float(data["pressure"]) - m.fs.state_block[1].pressure)**2)
    return expr*1e-7

### 4.3 Run the parameter estimation

We are now ready to set up the parameter estimation problem. We will create a parameter estimation object called pest. As shown below, we pass the method that returns an initialized model, data, variable_name, and the SSE expression to the Estimator method. tee=True will print the solver output after solving the parameter estimation problem.

In [9]:
pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee=True)

obj_value, parameters = pest.theta_est()

2022-08-25 12:01:01 [INFO] idaes.init.fs.state_block: Starting initialization
WARNING (W1002): Setting Var
    'fs.state_block[1].log_mole_frac_tbub[Vap,Liq,carbon_dioxide]' to a
    numeric value `4.301303339264284e-06` outside the bounds (None, 0).
    See also https://pyomo.readthedocs.io/en/latest/errors.html#w1002
2022-08-25 12:01:02 [INFO] idaes.init.fs.state_block: Dew and bubble point initialization: optimal - Optimal Solution Found.
2022-08-25 12:01:02 [INFO] idaes.init.fs.state_block: Equilibrium temperature initialization completed.
2022-08-25 12:01:02 [INFO] idaes.init.fs.state_block: State variable initialization completed.
2022-08-25 12:01:02 [INFO] idaes.init.fs.state_block: Phase equilibrium initialization: optimal - Optimal Solution Found.
2022-08-25 12:01:02 [INFO] idaes.init.fs.state_block: Property initialization: optimal - Optimal Solution Found.
2022-08-25 12:01:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.

2022-08-25 12:01:08 [INFO] idaes.init.fs.state_block: Dew and bubble point initialization: optimal - Optimal Solution Found.
2022-08-25 12:01:08 [INFO] idaes.init.fs.state_block: Equilibrium temperature initialization completed.
2022-08-25 12:01:08 [INFO] idaes.init.fs.state_block: State variable initialization completed.
2022-08-25 12:01:08 [INFO] idaes.init.fs.state_block: Phase equilibrium initialization: optimal - Optimal Solution Found.
2022-08-25 12:01:08 [INFO] idaes.init.fs.state_block: Property initialization: optimal - Optimal Solution Found.
2022-08-25 12:01:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-08-25 12:01:08 [INFO] idaes.init.fs.state_block: Starting initialization
WARNING (W1002): Setting Var
    'fs.state_block[1].log_mole_frac_tbub[Vap,Liq,carbon_dioxide]' to a
    numeric value `1.269889598521249e-06` outside the bounds (None, 0).
    See also https://pyomo.readthedocs.io/en/latest/errors.html#w1002

2022-08-25 12:01:14 [INFO] idaes.init.fs.state_block: State variable initialization completed.
2022-08-25 12:01:14 [INFO] idaes.init.fs.state_block: Phase equilibrium initialization: optimal - Optimal Solution Found.
2022-08-25 12:01:14 [INFO] idaes.init.fs.state_block: Property initialization: optimal - Optimal Solution Found.
2022-08-25 12:01:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-08-25 12:01:14 [INFO] idaes.init.fs.state_block: Starting initialization
WARNING (W1002): Setting Var
    'fs.state_block[1].log_mole_frac_tbub[Vap,Liq,carbon_dioxide]' to a
    numeric value `1.0835202436687703e-07` outside the bounds (None, 0).
    See also https://pyomo.readthedocs.io/en/latest/errors.html#w1002
2022-08-25 12:01:14 [INFO] idaes.init.fs.state_block: Dew and bubble point initialization: optimal - Optimal Solution Found.
2022-08-25 12:01:14 [INFO] idaes.init.fs.state_block: Equilibrium temperature initialization completed

## 5.0 Display results

Let us display the results by running the next cell.

In [10]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 29.282891

The values for the parameters are as follows:
fs.properties.PR_kappa[bmimPF6,carbon_dioxide] = -0.40714284003013756
fs.properties.PR_kappa[carbon_dioxide,bmimPF6] = 0.020593684002508987


Now we can use this parameters and include them in the configuration dictionary. We can also use `m.fs.properties = GenericParameterBlock(**configuration)` to solve unit models.